In [1]:
# 1. 파일 불러오기

import pandas as pd

temp_norm = pd.read_csv("C://Users//whs38//캡스톤//데이터//평균기온정규화.csv",encoding='euc-kr')
temp_norm.index
temp_norm = temp_norm.set_index("district")

rain_norm = pd.read_csv("C://Users//whs38//캡스톤//데이터//평균강수량정규화.csv",encoding='euc-kr')
rain_norm.index
rain_norm = rain_norm.set_index("district")

dust_norm = pd.read_csv("C://Users//whs38//캡스톤//데이터//대기오염정규화.csv",encoding='euc-kr')
dust_norm.index
dust_norm = dust_norm.set_index("district")

flt_pp_norm = pd.read_csv("C://Users//whs38//캡스톤//데이터//유동인구정규화.csv",encoding='euc-kr')
flt_pp_norm.index
flt_pp_norm = flt_pp_norm.set_index("district")

park_loc = pd.read_csv("C://Users//whs38//캡스톤//데이터//관광지.csv",encoding='euc-kr')
        
rest_loc = pd.read_csv("C://Users//whs38//캡스톤//데이터//음식점.csv",encoding='euc-kr')

    
# 2. 제주도 구별 경계선 데이터 불러오기
import json
geo_path = 'C://Users//whs38//캡스톤//jeju_map.geojson'
geo_str = json.load(open(geo_path, encoding='utf-8'))
    

# 3. 가중치 부여해서 하나로 합치기  

temp_norm = temp_norm*1      #int(RadioVariety.get())
rain_norm = rain_norm*1      #int(RadioVariety2.get())
dust_norm = dust_norm*1      #int(RadioVariety3.get())
flt_pp_norm = flt_pp_norm*1  #int(RadioVariety4.get())

total = pd.concat([temp_norm, rain_norm, dust_norm, flt_pp_norm]) 
total_final = total.groupby(by=['district'], as_index=True).sum()
    
         
# 4. 지도에 시각화 하기 
import folium
total_map = folium.Map(location=[33.38, 126.5], zoom_start=11,
                           tiles='Stamen Terrain') 

# 기온 + 강수량 + 대기오염도 + 유동인구
legend = "test" + "(가중치 : 기온 " + "1" + ", 강수량 " + "1" + \
        ", 대기오염도 " + "1"+ ", 유동인구 " + "1" + ")"

total_map.choropleth(geo_data = geo_str,
                    data = total_final["6월"],   #  combobox.get()  
                    columns = [total_final.index, total_final["6월"]], # combobox.get()
                    fill_color = 'PuRd', #PuRd, YlGnBu
                    key_on = 'feature.properties.EMD_NM',
                    name = "기온+강수량+대기오염도+유동인구",
                    legend_name = legend)

# 공원 MarkerCluster
from folium.plugins import MarkerCluster

p_lat = park_loc.lat.values
p_lng = park_loc.lng.values
p_locations = list(zip(p_lat,p_lng))
p_popups = list(park_loc.name.values)
p_icons = [folium.Icon(icon="tree", prefix="fa", color="green") for _ in range(len(p_locations))]

p_icon_create_function = """\
function(cluster) {
    return L.divIcon({
    html: '<b>' + cluster.getChildCount() + '</b>',
    className: 'marker-cluster marker-cluster-small',
    iconSize: new L.Point(30, 30)
    });
}"""

p_marker_cluster = MarkerCluster(
    locations=p_locations, popups=p_popups,
    name='공원',
    overlay=True,
    control=True,
    icon_create_function = p_icon_create_function,
    icons = p_icons)

p_marker_cluster.add_to(total_map)
 
# 맛집 위치
r_lat = rest_loc.lat.values
r_lng = rest_loc.lng.values
r_locations = list(zip(r_lat,r_lng))
r_popups = list(rest_loc.restaurant.values)
r_icons = [folium.Icon(icon="heart", prefix="fa", color="orange") for _ in range(len(r_locations))]

r_icon_create_function = """\
function(cluster) {
    return L.divIcon({
    html: '<b>' + cluster.getChildCount() + '</b>',
    className: 'marker-cluster marker-cluster-large',
    iconSize: new L.Point(30, 30)
    });
}"""

r_marker_cluster = MarkerCluster(
    locations=r_locations, popups=r_popups,
    name='맛집',
    overlay=True,
    control=True,
    icon_create_function = r_icon_create_function,
    icons = r_icons)


r_marker_cluster.add_to(total_map)

save_path ="C://Users//whs38//캡스톤//시각화" + "overlap" + ".html"    #str(combobox.get())
total_map.save(save_path)

    





C:\Users\whs38\anaconda3\lib\site-packages\folium\folium.py:409: FutureWarning: The choropleth  method has been deprecated. Instead use the new Choropleth class, which has the same arguments. See the example notebook 'GeoJSON_and_choropleth' for how to do this.
  warnings.warn(
